# Generating Qubit Hamiltonians

In [16]:
from utility import *
from functools import reduce

Specify the Qubit Hamiltonian of a molecule by its name, internuclear distances, basis set, and fermion-to-qubit transformation.
Here, we show the resulting Hamiltonian for $H_2$ in STO-3G with $1\overset{\circ}{A}$ between the $H$ atoms. 

In [17]:
# Identity 
I = np.asarray([[1.,0.],[0.,1.]])
# Pauli X
X = np.asarray([[0.,1.],[1.,0.]])
# Pauli Y
Y = np.asarray([[0.,-1j],[1j,0.]])
# Pauli Z
Z = np.asarray([[1, 0], [0, -1]])

In [18]:
def operatorfromstring(pauli_string, max_orbitals):
    """
    Generate a many-body operator from a list of single-qubit 
    pauli matrices.
    ---------------------------------------------------------
    Input    pauli_string: str , (ex: [X,Z,X,Y,...])
    
    Output   pauli_op: np.array, shape = [2**N,2**N]
    """
    op_list = []
    eye_num = 0
    
    for k in range(len(pauli_string)):
        if (pauli_string[k][1] == 'X'):
            op_list.append(X)
        elif (pauli_string[k][1] == 'Y'):
            op_list.append(Y)
        elif (pauli_string[k][1] == 'Z'):
            op_list.append(Z)
        else:
            op_list.append(I)
            
    if not op_list:
            op_list.append(I)
            
    if len(op_list) < max_orbitals:
        eye_num = max_orbitals - len(op_list)
        for e in range(eye_num):
            op_list.append(I)         

    return reduce(np.kron,op_list)

def hamiltonianQ(pauli_list, interactions, max_orbitals):
    """
    Generate a many-body hamiltonian
    ---------------------------------------------------------
    Input    pauli_list: list of pauli strings
             interactions: list of interaction strengths
    
    Output   hamiltonian: np.array, shape = [2**N,2**N]
    """
    N = max_orbitals
    hamiltonian = np.zeros((1<<N,1<<N),dtype=complex)
    for i,pauli in enumerate(pauli_list):
        
        hamiltonian += interactions[i]*operatorfromstring(pauli, N)
    return hamiltonian

def eigensolve(hamiltonian):
    """
    Compute ground state energy and wavefunction
    """
    (eigenvalues,eigenstates) = np.linalg.eigh(hamiltonian)
    return eigenvalues[0],eigenstates[:,0]

In [19]:
qubit_transf = 'jw' # Jordan-Wigner transformations
h2 = get_qubit_hamiltonian(mol='h2', geometry=1, basis='sto3g', qubit_transf=qubit_transf)
print(h2)

-0.32760818967480887 [] +
-0.04919764587136755 [X0 X1 Y2 Y3] +
0.04919764587136755 [X0 Y1 Y2 X3] +
0.04919764587136755 [Y0 X1 X2 Y3] +
-0.04919764587136755 [Y0 Y1 X2 X3] +
0.13716572937099508 [Z0] +
0.15660062488237947 [Z0 Z1] +
0.10622904490856075 [Z0 Z2] +
0.15542669077992832 [Z0 Z3] +
0.13716572937099503 [Z1] +
0.15542669077992832 [Z1 Z2] +
0.10622904490856075 [Z1 Z3] +
-0.13036292057109117 [Z2] +
0.16326768673564346 [Z2 Z3] +
-0.13036292057109117 [Z3]


Alternatively, the qubit-tapering technique can find a smaller effective Hamitlonian by subsitituting operators with $\pm 1$. This technique is detailed in Bravyi's work ([Bravyi et al., "Tapering off qubits to simulate fermionic Hamiltonians", arXiv:1701.08213](https://arxiv.org/abs/1701.08213)). 

In [20]:
print("The effective Hamiltonian:\n {}".format(taper_hamiltonian(h2, n_spin_orbitals=12, n_electrons=2, qubit_transf=qubit_transf))) 

The effective Hamiltonian:
 -0.5310513494337641 [] +
0.1967905834854702 [X0] +
-0.5350572998841725 [Z0]


We can verify that this new Hamiltonian indeed includes the ground state. 

In [22]:
print("The ground state energy:")
obtain_PES('h2', [1], 'sto-3g', 'fci')

# Building the matrix representation of the effective Hamiltonian
hamiltonian = taper_hamiltonian(h2, n_spin_orbitals=12, n_electrons=2, qubit_transf=qubit_transf)
pauli_list = []
interaction_list = []
max_orbitals = 0
#Generate pauli operations list and coefficient list(interactions)
for pauli, interaction in hamiltonian.terms.items():
    pauli_list.append(pauli)
    interaction_list.append(interaction)

#Identify Number Max Oribitals    
for x in list(hamiltonian.terms.keys()):
    _max = 0
    for y in x:
        if y[0] > _max:
            _max = y[0]
max_orbitals = _max + 1


h2_matrix = hamiltonianQ(pauli_list, interaction_list, max_orbitals)

# Obtain the eigenvalues
eigvals, _ = np.linalg.eigh(h2_matrix)

print("\nThe eigenvalues in the effective Hamiltonian: \n {}".format(eigvals))

# Obtain the eigenvalues
#eigvals, _ = np.linalg.eigh(h2_matrix)
#print("\nThe eigenvalues in the effective Hamiltonian: \n {}".format(eigvals))

#print(hamiltonian.terms)
#len(list(hamiltonian.terms.keys())[1])
#max(len(x) for x in list(hamiltonian.terms.keys())) 

#list(hamiltonian.terms.keys())[9]
#print(hamiltonian.terms.get(()))
#print(pauli_list )
#print(operatorfromstring(pauli_list[2], max_orbitals))
#print(len(pauli_list[2]))
#eye_list = []
#eye_num = max_orbitals - len(pauli_list[2])

#for e in range(eye_num):
  #  eye_list.append(I)

#reduce(np.kron,eye_list)

The ground state energy:
E = -1.1011503301329566 Eh

The eigenvalues in the effective Hamiltonian: 
 [-1.10115033  0.03904763]


# LiH

In [ ]:
#########Assumptions made
#Made the assumption based on energy graphs from last notebook
#The hamiltonian outputs the number qubits needed to represent the molecule 
#mapping fermonic states to qubit states --> Qubit representation of hamiltonian
#Jordon-Wigner --> Fermion to Qubit Mapping
qubit_transf = 'jw' # Jordan-Wigner transformations
lih = get_qubit_hamiltonian(mol='lih', geometry=2, basis='sto3g', qubit_transf=qubit_transf)
print(lih)

In [ ]:
#########Assumptions made
#
print("The effective Hamiltonian:\n {}".format(taper_hamiltonian(lih, n_spin_orbitals=12, n_electrons=4, qubit_transf=qubit_transf))) 

In [ ]:
hamiltonian = taper_hamiltonian(lih, n_spin_orbitals=12, n_electrons=4, qubit_transf=qubit_transf)
#test = list(hamiltonian)[200]
#print(test)
#test = str(test)
#print(test.split())
#test = test[test.find('[')+1 : test.find(']')]
#test.split()
pauli_list = []
interaction_list = []
for pauli, interaction in hamiltonian.terms.items():
    pauli_list.append(pauli)
    interaction_list.append(interaction)
    
#print(pauli_list[1])
#print(interaction_list[1])
#operatorfromstring(pauli_list[1])
H = hamiltonianQ(pauli_list,interaction_list)

In [ ]:
Z

In [ ]:
print("The ground state energy:")
obtain_PES('lih', [1], 'sto-3g', 'fci')

# Building the matrix representation of the effective Hamiltonian
I, X, Z = np.identity(2), np.array([[0, 1], [1, 0]]), np.array([[1, 0], [0, -1]])
h2_matrix = -0.53105134 * I + 0.19679058 * X - 0.53505729 * Z

# Obtain the eigenvalues
eigvals, _ = np.linalg.eigh(h2_matrix)
print("\nThe eigenvalues in the effective Hamiltonian: \n {}".format(eigvals))

# H4

In [ ]:
#########Assumptions made
#Chose geometry value from ground state energy aproximation in first notebook 86 something close to a local min

qubit_transf = 'jw' # Jordan-Wigner transformations
h4 = get_qubit_hamiltonian(mol='h4', geometry=86, basis='sto3g', qubit_transf=qubit_transf)
print(h4)

In [ ]:
print("The effective Hamiltonian:\n {}".format(taper_hamiltonian(h4, n_spin_orbitals=8, n_electrons=220, qubit_transf=qubit_transf))) 